In [ ]:
#instalation libs

!pip3 install yfinance
!pip3 install crewai
!pip3 install 'crewai[tools]'
!pip3 install langchain
!pip3 install langchain-openai

In [7]:
import json
import os
from datetime import datetime

import yfinance as yf

from crewai import Agent, Task

from langchain.tools import Tool
from langchain_openai import ChatOpenAI


In [3]:
#CREATE YAHOO FINANCE TOOL

def fetch_stock_price(ticket):
    stock = yf.download(ticket, start="2023-08-08", end="2024-08-08")
    return stock

yahoo_finance_tool = Tool(
    name = "Yahoo Finance Tool",
    description = "Fetches stocks prices for {ticket} from the last year about a specfic company from Yahoo Finance API",
    func= lambda ticket: fetch_stock_price(ticket)
)


In [4]:
response = yahoo_finance_tool.run("AAPL")

[*********************100%%**********************]  1 of 1 completed


In [ ]:
print(response)